### 1. Setup

In [1]:
%autoawait
import time
import asyncio
from termcolor import colored,cprint

from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_BASE = ""

WEBHOOK_PORT = 8052
ADMIN_URL = "http://manufacturer-agent:8051"

# Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(admin_url=ADMIN_URL)




IPython autoawait is `on`, and set to use `asyncio`


In [2]:
agent_controller.init_webhook_server(webhook_host=WEBHOOK_HOST,
                                     webhook_port=WEBHOOK_PORT,
                                     webhook_base=WEBHOOK_BASE)

In [3]:
loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

def proof_handler(payload):
    print("Handle present proof")
    role = payload["role"]
    pres_ex_id = payload["presentation_exchange_id"]
    state = payload["state"]
    print(f"Role {role}, Exchange {pres_ex_id} in state {state}")

proof_listener = {
    "topic": "present_proof",
    "handler": proof_handler
}
agent_controller.register_listeners([proof_listener], defaults=True)

Subscribing too: present_proof


### 2. Go to seller notebook to setup

### 5. Check Connection

In [4]:
try:
    response = await agent_controller.connections.get_connections()
    results = response['results']
    print("Results : ", results)
    print('\n')
    if len(results) > 0:
        connection = response['results'][0]
        print("Connection :", connection)
        if connection['state'] == 'active':
            connection_id = connection["connection_id"]
            print("\nActive Connection ID : ", connection_id)
        else:
            print("\nNo active connection found - wait a bit and execute again")
    else:
        print("You must create a connection")
except ConnectionRefusedError as e:
    print(repr(e))

Results :  [{'invitation_mode': 'once', 'invitation_key': 'EUWUf4SDMdzmZDxBgCkyHf8fE8Fs3925Y4yQWjVHt4bV', 'accept': 'manual', 'my_did': '5oeYVJAXiht1YQ6ZmA1cwr', 'their_role': 'inviter', 'request_id': '6ce46cb4-b21f-4b41-befa-b7c50ab478f4', 'connection_id': '801fb919-0535-4e41-bfca-f9bd098c2f16', 'created_at': '2023-01-04 03:50:27.747959Z', 'their_did': 'RvW6EJ715wZivuiM9o2oZe', 'updated_at': '2023-01-04 03:50:47.993152Z', 'their_label': 'Bob', 'rfc23_state': 'completed', 'routing_state': 'none', 'state': 'active'}]


Connection : {'invitation_mode': 'once', 'invitation_key': 'EUWUf4SDMdzmZDxBgCkyHf8fE8Fs3925Y4yQWjVHt4bV', 'accept': 'manual', 'my_did': '5oeYVJAXiht1YQ6ZmA1cwr', 'their_role': 'inviter', 'request_id': '6ce46cb4-b21f-4b41-befa-b7c50ab478f4', 'connection_id': '801fb919-0535-4e41-bfca-f9bd098c2f16', 'created_at': '2023-01-04 03:50:27.747959Z', 'their_did': 'RvW6EJ715wZivuiM9o2oZe', 'updated_at': '2023-01-04 03:50:47.993152Z', 'their_label': 'Bob', 'rfc23_state': 'completed'

### 6. Create verify request

In [5]:
import time

response = await agent_controller.wallet.get_public_did()
print(response)
issuer_did = response["result"]["did"]

print("Request proof of the device")
#Set some variables

revocation = False
exchange_tracing = False

#Enable this to ask for attributes to identity a user
req_attrs = [
    {"name": "ProductionDate", "restrictions": [{"issuer_did": issuer_did}]},
    {"name": "Manufacturer", "restrictions": [{"issuer_did": issuer_did}]},
    {"name": "Owner", "restrictions": [{"issuer_did": issuer_did}]},
    {"name": "Type", "restrictions": [{"issuer_did": issuer_did}]}
]


curtime = int(time.time())
#Set predicates for Zero Knowledge Proofs
req_preds = [
    # test zero-knowledge proofs
    {
        "name": "ExpirationDate",
        "p_type": ">=",
        "p_value": curtime,
        "restrictions": [{"issuer_did": issuer_did}],
    },
    {
        "name": "Status",
        "p_type": ">",
        "p_value": 0,
        "restrictions": [{"issuer_did": issuer_did}],
    }
]

indy_proof_request = {
    "name": "Proof of Personal Information",
    "version": "1.0",
    "requested_attributes": {
        f"0_{req_attr['name']}_uuid":
        req_attr for req_attr in req_attrs
    },
    "requested_predicates": {
        f"0_{req_pred['name']}_GE_uuid":
        req_pred for req_pred in req_preds
    },
}
indy_proof_request["non_revoked"] = {"to": int(time.time())}

#proof_request = indy_proof_request
exchange_tracing_id = exchange_tracing
proof_request_web_request = {
    "connection_id": connection_id,
    "proof_request": indy_proof_request,
    "trace": exchange_tracing,
}

{'result': {'did': 'PQRXDxdGqQGSZ8z69p4xZP', 'verkey': 'DDEKJtBjzaXAJtpwetdPiXH5s4rNUEzG18V15QoLRcZZ', 'posture': 'public'}}
Request proof of the device


### 7. Send Request

In [6]:
response = await agent_controller.proofs.send_request(proof_request_web_request)
print(response)
presentation_exchange_id = response['presentation_exchange_id']
print("\n")
print(presentation_exchange_id)

{'thread_id': '9f88ce22-6a2b-436d-b51a-64aff2ae437e', 'presentation_exchange_id': '6cc42679-b31b-44b9-a172-ed16eb17974b', 'initiator': 'self', 'presentation_request': {'name': 'Proof of Personal Information', 'version': '1.0', 'requested_attributes': {'0_ProductionDate_uuid': {'name': 'ProductionDate', 'restrictions': [{'issuer_did': 'PQRXDxdGqQGSZ8z69p4xZP'}]}, '0_Manufacturer_uuid': {'name': 'Manufacturer', 'restrictions': [{'issuer_did': 'PQRXDxdGqQGSZ8z69p4xZP'}]}, '0_Owner_uuid': {'name': 'Owner', 'restrictions': [{'issuer_did': 'PQRXDxdGqQGSZ8z69p4xZP'}]}, '0_Type_uuid': {'name': 'Type', 'restrictions': [{'issuer_did': 'PQRXDxdGqQGSZ8z69p4xZP'}]}}, 'requested_predicates': {'0_ExpirationDate_GE_uuid': {'name': 'ExpirationDate', 'p_type': '>=', 'p_value': 1672820152, 'restrictions': [{'issuer_did': 'PQRXDxdGqQGSZ8z69p4xZP'}]}, '0_Status_GE_uuid': {'name': 'Status', 'p_type': '>', 'p_value': 0, 'restrictions': [{'issuer_did': 'PQRXDxdGqQGSZ8z69p4xZP'}]}}, 'non_revoked': {'to': 16728

### 8. Go to seller notebook to send verify response

### 11. Get verify response

In [7]:
verify = await agent_controller.proofs.verify_presentation(presentation_exchange_id)
print(verify)

Handle present proof
Role verifier, Exchange 0a5aa814-3293-4494-8ccb-72bb5bd23cdc in state verified
{'thread_id': 'dc025a05-7694-4ac6-945b-5def4836e8b6', 'presentation_exchange_id': '0a5aa814-3293-4494-8ccb-72bb5bd23cdc', 'initiator': 'self', 'presentation_request': {'name': 'Proof of Personal Information', 'version': '1.0', 'requested_attributes': {'0_ProductionDate_uuid': {'name': 'ProductionDate', 'restrictions': [{'issuer_did': 'PQRXDxdGqQGSZ8z69p4xZP'}]}, '0_Manufacturer_uuid': {'name': 'Manufacturer', 'restrictions': [{'issuer_did': 'PQRXDxdGqQGSZ8z69p4xZP'}]}, '0_Owner_uuid': {'name': 'Owner', 'restrictions': [{'issuer_did': 'PQRXDxdGqQGSZ8z69p4xZP'}]}, '0_Type_uuid': {'name': 'Type', 'restrictions': [{'issuer_did': 'PQRXDxdGqQGSZ8z69p4xZP'}]}}, 'requested_predicates': {'0_ExpirationDate_GE_uuid': {'name': 'ExpirationDate', 'p_type': '>=', 'p_value': 1672805697, 'restrictions': [{'issuer_did': 'PQRXDxdGqQGSZ8z69p4xZP'}]}, '0_Status_GE_uuid': {'name': 'Status', 'p_type': '>', 'p_

### 12. Check the verfiy state. If credential revoked, verify['verified'] should become Fales

In [8]:
print(verify['state'])
print(verify['state'] == 'verified')
print(verify['verified'])

verified
True
true


### 13. Get some revealed attribute

In [9]:
for (name, val) in verify['presentation']['requested_proof']['revealed_attrs'].items():
    ## This is the actual data that you want. It's a little hidden
    print(name + " : " + val['raw'])

0_Manufacturer_uuid : GOOD_COMPANY
0_Type_uuid : Dentistry
0_Owner_uuid : GOOD_SELLER
0_ProductionDate_uuid : 1672804394


### **Terminate**

In [10]:
response = await agent_controller.terminate()